<a href="https://colab.research.google.com/github/mirnanoukari/FoR/blob/main/Assignment4/Dynamics_FoR_A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Fundamentals of Robotics**
## Assignment 4: *Dynamics*

Work of: Mirna Alnoukari

Email: m.alnoukari@innopolis.university



## ***Introduction***
In this assignment we are going to derive the dynamic model for our 6-DoF antropomorphic arm with a spherical wrist, with Euler-Lagrange method:    
$$
M(q) · q̈ + C(q, q̇)· q̇ + g(q) = τ
$$

The dynamics of a robot are used to calculate torques, thus achieving a specific trajectory $(q, q̇, q̈)$ which are the input to a robot control system.


The structure of this code:

All the components in the dynamic equation are obtained symbolically using `Sympy`:
1.   Kinematics calculations taken from previous assignment
2.   Center of mass function that will be needed for calculation of jacobian matrices
3.   Linear and angular jacobian matrices calculations that will be used for deriving matrix of inertia
4.   Inertia matrix
5.   Corioli matrix
6.   Gravity matrix

And lastly: 
*   Functions for numerical input values




## Kinematics
First, the following cell is taken from the previous kinematics assignement to calculate rotation and translation matrices:
 

In [ ]:
import numpy as np
from numpy.linalg import inv
import sympy as sp

def Rx(q):
   return np.array([[1, 0, 0, 0],
              [0, sp.cos(q), - sp.sin(q), 0],
              [0, sp.sin(q), sp.cos(q), 0],
              [0, 0, 0, 1]])
   

def Ry (q):
  return np.array([[sp.cos(q), 0, sp.sin(q), 0],
              [0, 1, 0, 0],
              [- sp.sin(q), 0, sp.cos(q), 0],
              [0, 0, 0, 1]])

def Rz(q):
  return np.array([[sp.cos(q), - sp.sin(q), 0, 0],
              [sp.sin(q), sp.cos(q), 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tx(d):
   return np.array([[1, 0, 0, d],
              [0, 1, 0, 0],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])
   
def Ty(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, d],
              [0, 0, 1, 0],
              [0, 0, 0, 1]])

def Tz(d):
  return np.array([[1, 0, 0, 0],
              [0, 1, 0, 0],
              [0, 0, 1, d],
              [0, 0, 0, 1]])


The following are the parameters we will use later to obtain every component of the dynamic equation, which are M, C, and g matrices.
q and dq are kept symbolically while we assumed simple values for the other variables to simplify our calculations.

In [ ]:
q = sp.symbols('q1 q2 q3 q4 q5 q6')
dq = sp.symbols('dq1 dq2 dq3 dq4 dq5 dq6')
lc = [0.5 for _ in range(6)]
l = [1 for _ in range(6)]
I = np.array([[1 for b in ['x', 'y', 'z']] for a in ['x', 'y', 'z']])
m = [1 for _ in range(6)]

## **Center of Mass**
To be able to derive the Jacobian matrix for linear and angular velocity, we need to calculate the center of mass of each link in our 6-DoF robot. We can do this by getting the position for each link from our forward kinematics configuration:
$$
T = T_{base}\cdot R_z^{q_1}\space T_z^{l_1} \cdot R_x^{q_2}\space T_z^{l_2} \cdot R_x^{q_3}\space T_z^{l_3} \cdot T_z^{l_4} \space R_z^{q_4} \cdot R_y^{q_5} \cdot R_z^{q_6} \space T_z^{l_5} \space T_z^{l_6} \cdot T_{tool}
$$

In [ ]:
com = np.zeros((6, 4, 4), dtype=np.object)
com[0] = Rz(q[0]) @ Tz(lc[0])
com[1] = Rz(q[0]) @ Tz(l[0]) @ Rx(q[1]) @ Tz(lc[1])
com[2] = Rz(q[0]) @ Tz(l[0]) @ Rx(q[1]) @ Tz(l[1]) @ Rx(q[2]) @ Tz(lc[2])
com[3] = Rz(q[0]) @ Tz(l[0]) @ Rx(q[1]) @ Tz(l[1]) @ Rx(q[2]) @ Tz(l[2]) @ Rz(q[3]) @ Tz(lc[3])
com[4] = Rz(q[0]) @ Tz(l[0]) @ Rx(q[1]) @ Tz(l[1]) @ Rx(q[2]) @ Tz(l[2]) @ Rz(q[3]) @ Tz(l[3]) @ Ry(q[4]) @ Tz(lc[4])
com[5] = Rz(q[0]) @ Tz(l[0]) @ Rx(q[1]) @ Tz(l[1]) @ Rx(q[2]) @ Tz(l[2]) @ Rz(q[3]) @ Tz(l[3]) @ Ry(q[4]) @ Tz(l[4]) @ Rz(q[5]) @ Tz(lc[5])

sp.Array(com)

[[[1.0*cos(q1), -1.0*sin(q1), 0, 0], [1.0*sin(q1), 1.0*cos(q1), 0, 0], [0.0, 0.0, 1.0, 0.5], [0.0, 0.0, 0.0, 1.0]], [[1.0*cos(q1), -1.0*sin(q1)*cos(q2), 1.0*sin(q1)*sin(q2), 0.5*sin(q1)*sin(q2)], [1.0*sin(q1), 1.0*cos(q1)*cos(q2), -1.0*sin(q2)*cos(q1), -0.5*sin(q2)*cos(q1)], [0, 1.0*sin(q2), 1.0*cos(q2), 0.5*cos(q2) + 1.0], [0, 0, 0, 1.0]], [[1.0*cos(q1), 1.0*sin(q1)*sin(q2)*sin(q3) - 1.0*sin(q1)*cos(q2)*cos(q3), 1.0*sin(q1)*sin(q2)*cos(q3) + 1.0*sin(q1)*sin(q3)*cos(q2), 0.5*sin(q1)*sin(q2)*cos(q3) + 1.0*sin(q1)*sin(q2) + 0.5*sin(q1)*sin(q3)*cos(q2)], [1.0*sin(q1), -1.0*sin(q2)*sin(q3)*cos(q1) + 1.0*cos(q1)*cos(q2)*cos(q3), -1.0*sin(q2)*cos(q1)*cos(q3) - 1.0*sin(q3)*cos(q1)*cos(q2), -0.5*sin(q2)*cos(q1)*cos(q3) - 1.0*sin(q2)*cos(q1) - 0.5*sin(q3)*cos(q1)*cos(q2)], [0, 1.0*sin(q2)*cos(q3) + 1.0*sin(q3)*cos(q2), -1.0*sin(q2)*sin(q3) + 1.0*cos(q2)*cos(q3), -0.5*sin(q2)*sin(q3) + 0.5*cos(q2)*cos(q3) + 1.0*cos(q2) + 1.0], [0, 0, 0, 1.0]], [[1.0*(sin(q1)*sin(q2)*sin(q3) - sin(q1)*cos(q2)*cos

## **Jacobian Matrices**
The Jacobian matrix for **linear velocity** $(3,n)$:
$$
J^i_v = \left(
\begin{array}{cccc}
\frac{δx_i}{δq_1}  & \frac{δx_i}{δq_2} & ... & \frac{δx_i}{δq_n}\\ 
\frac{δy_i}{δq_1}  & \frac{δy_i}{δq_2} & ... & \frac{δy_i}{δq_n}\\
\frac{δz_i}{δq_1}  & \frac{δz_i}{δq_2} & ... & \frac{δz_i}{δq_n}
\end{array}
\right)
$$

In [ ]:
Jv = np.zeros((6,3,6), dtype=np.object)
for n in range(6):
  for i in range(3):
    for j in range(6):
      Jv[n, i, j] = sp.diff(com[n, i, 3], q[j]) 

sp.Array(Jv)

[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]], [[0.5*sin(q2)*cos(q1), 0.5*sin(q1)*cos(q2), 0, 0, 0, 0], [0.5*sin(q1)*sin(q2), -0.5*cos(q1)*cos(q2), 0, 0, 0, 0], [0, -0.5*sin(q2), 0, 0, 0, 0]], [[0.5*sin(q2)*cos(q1)*cos(q3) + 1.0*sin(q2)*cos(q1) + 0.5*sin(q3)*cos(q1)*cos(q2), -0.5*sin(q1)*sin(q2)*sin(q3) + 0.5*sin(q1)*cos(q2)*cos(q3) + 1.0*sin(q1)*cos(q2), -0.5*sin(q1)*sin(q2)*sin(q3) + 0.5*sin(q1)*cos(q2)*cos(q3), 0, 0, 0], [0.5*sin(q1)*sin(q2)*cos(q3) + 1.0*sin(q1)*sin(q2) + 0.5*sin(q1)*sin(q3)*cos(q2), 0.5*sin(q2)*sin(q3)*cos(q1) - 0.5*cos(q1)*cos(q2)*cos(q3) - 1.0*cos(q1)*cos(q2), 0.5*sin(q2)*sin(q3)*cos(q1) - 0.5*cos(q1)*cos(q2)*cos(q3), 0, 0, 0], [0, -0.5*sin(q2)*cos(q3) - 1.0*sin(q2) - 0.5*sin(q3)*cos(q2), -0.5*sin(q2)*cos(q3) - 0.5*sin(q3)*cos(q2), 0, 0, 0]], [[1.5*sin(q2)*cos(q1)*cos(q3) + 1.0*sin(q2)*cos(q1) + 1.5*sin(q3)*cos(q1)*cos(q2), -1.5*sin(q1)*sin(q2)*sin(q3) + 1.5*sin(q1)*cos(q2)*cos(q3) + 1.0*sin(q1)*cos(q2), -1.5*sin(q1)*sin(q2)*sin(q3) + 1.5*sin(q1)

The Jacobian matrix for **angular velocity** $(3,n)$:
$$
\begin{align}
J_w^1 &= \left(
\begin{array}{cccc}
 u_0 & 0 & 0 & \cdots & 0 
\end{array}
\right)\\
J_w^2 &= \left(
\begin{array}{cccc}
 u_0 & u_1 & 0 & \cdots & 0 
\end{array}
\right)\\
J_w^3 &= \left(
\begin{array}{cccc}
 u_0 & u_1 & u_2 & 0 & \cdots & 0 
\end{array}
\right)\\
\vdots \\
J_w^n &= \left(
\begin{array}{cccc}
 u_0 & u_1 & u_2 & ... & u_n
\end{array}
\right)\\
\end{align}
$$

In [ ]:
u = [np.array([0, 0, 1]), com[0, :3, 0], com[1, :3, 0], com[2, :3, 2], com[3, :3, 1], com[4, :3, 2]]
Jw = np.zeros((6,3,6), dtype=np.object)
for n in range(6):
  for j in range(n + 1):
    Jw[n, :, j] = u[j]

sp.Array(Jw)

[[[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0]], [[0, 1.0*cos(q1), 0, 0, 0, 0], [0, 1.0*sin(q1), 0, 0, 0, 0], [1, 0.0, 0, 0, 0, 0]], [[0, 1.0*cos(q1), 1.0*cos(q1), 0, 0, 0], [0, 1.0*sin(q1), 1.0*sin(q1), 0, 0, 0], [1, 0.0, 0, 0, 0, 0]], [[0, 1.0*cos(q1), 1.0*cos(q1), 1.0*sin(q1)*sin(q2)*cos(q3) + 1.0*sin(q1)*sin(q3)*cos(q2), 0, 0], [0, 1.0*sin(q1), 1.0*sin(q1), -1.0*sin(q2)*cos(q1)*cos(q3) - 1.0*sin(q3)*cos(q1)*cos(q2), 0, 0], [1, 0.0, 0, -1.0*sin(q2)*sin(q3) + 1.0*cos(q2)*cos(q3), 0, 0]], [[0, 1.0*cos(q1), 1.0*cos(q1), 1.0*sin(q1)*sin(q2)*cos(q3) + 1.0*sin(q1)*sin(q3)*cos(q2), 1.0*(sin(q1)*sin(q2)*sin(q3) - sin(q1)*cos(q2)*cos(q3))*cos(q4) - 1.0*sin(q4)*cos(q1), 0], [0, 1.0*sin(q1), 1.0*sin(q1), -1.0*sin(q2)*cos(q1)*cos(q3) - 1.0*sin(q3)*cos(q1)*cos(q2), 1.0*(-sin(q2)*sin(q3)*cos(q1) + cos(q1)*cos(q2)*cos(q3))*cos(q4) - 1.0*sin(q1)*sin(q4), 0], [1, 0.0, 0, -1.0*sin(q2)*sin(q3) + 1.0*cos(q2)*cos(q3), 1.0*(sin(q2)*cos(q3) + sin(q3)*cos(q2))*cos(q4), 0]], [[0, 1.0*cos(q1), 

## **Inetria Matrix**
Now we are able to calculate matrix $M (n, n)$ from the formula:

$$
M = \sum_{i=1}^n m_i J_v^{i^T} J_v^i + J_w^{i^T} R_i I_i R_i^T J_w^i
$$


In [ ]:
R = np.zeros((6, 3, 3), dtype=np.object)
for n in range(6):
  R[n] = com[n, :3, :3]

M = np.zeros((6, 6), dtype=np.object)
for i in range(6):
  M += m[i] * Jv[i].T @ Jv[i] + Jw[i].T @ R[i] @ I @ R[i].T @ Jw[i]

# too large to print

## **Coriolis Matrix**
$$
C = \sum_{i=1}^n \sum_{j=1}^n \sum_{k=1}^n \frac{1}{2} (\frac{\delta M_{i,j}}{δM_k} + \frac{\delta M_{i,k}}{δM_j} - \frac{\delta M_{j,k}}{δM_i})
$$
The dmensions of this matrix are $(n,n)$ and the output is implemented in code but printing it took too long

In [ ]:
C = np.zeros((6, 6), dtype=np.object)
for i in range(6):
  for j in range(6):
    for k in range(6):
      cijk = 0.5 * (sp.diff(M[i,j], q[k]) + sp.diff(M[i,k], q[j]) - sp.diff(M[j,k], q[i]))
      C[i, j] += cijk * dq[k]

# too large to print

## **Gravity Matrix**
The last matrix $G (1,6)$ can be calculated with the following formula:
$$
G = - \sum_{i=1}^n \sum_{k=1}^n J_{vi}^{k^T} m_k g_0
$$

In [ ]:
g = np.zeros(6, dtype=np.object)
for i in range(6):
  for k in range(6):
    g[i] -= np.dot(Jv[k, :, i], m[k] * np.array([0, 0, 9.81]))

sp.Array(g)

[0, -9.81*(-1.5*sin(q2)*sin(q3) + 1.5*cos(q2)*cos(q3))*sin(q4)*sin(q5) - 9.81*(-0.5*sin(q2)*sin(q3) + 0.5*cos(q2)*cos(q3))*sin(q4)*sin(q5) - 9.81*(-1.5*sin(q2)*cos(q3) - 1.5*sin(q3)*cos(q2))*cos(q5) - 9.81*(-0.5*sin(q2)*cos(q3) - 0.5*sin(q3)*cos(q2))*cos(q5) + 58.86*sin(q2)*cos(q3) + 44.145*sin(q2) + 58.86*sin(q3)*cos(q2), -9.81*(-1.5*sin(q2)*sin(q3) + 1.5*cos(q2)*cos(q3))*sin(q4)*sin(q5) - 9.81*(-0.5*sin(q2)*sin(q3) + 0.5*cos(q2)*cos(q3))*sin(q4)*sin(q5) - 9.81*(-1.5*sin(q2)*cos(q3) - 1.5*sin(q3)*cos(q2))*cos(q5) - 9.81*(-0.5*sin(q2)*cos(q3) - 0.5*sin(q3)*cos(q2))*cos(q5) + 58.86*sin(q2)*cos(q3) + 58.86*sin(q3)*cos(q2), -9.81*(0.5*sin(q2)*cos(q3) + 0.5*sin(q3)*cos(q2))*sin(q5)*cos(q4) - 9.81*(1.5*sin(q2)*cos(q3) + 1.5*sin(q3)*cos(q2))*sin(q5)*cos(q4), 9.81*(-1.5*sin(q2)*sin(q3) + 1.5*cos(q2)*cos(q3))*sin(q5) + 9.81*(-0.5*sin(q2)*sin(q3) + 0.5*cos(q2)*cos(q3))*sin(q5) - 9.81*(0.5*sin(q2)*cos(q3) + 0.5*sin(q3)*cos(q2))*sin(q4)*cos(q5) - 9.81*(1.5*sin(q2)*cos(q3) + 1.5*sin(q3)*cos(q2))*s

## **Numerical Input**
The following functions take numerical values for $q$ , $q̇$ from the user and subtitue them in the symbolic equations we derived before:

In [ ]:
def Mnum(qVal):
  ret = np.zeros((6, 6))
  for i in range(6):
    for j in range(6):
      ret[i, j] = M[i, j].subs([(q[i], qVal[i]) for i in range(6)]).evalf()
  return ret

def Cnum(qVal, dqVal):
  ret = np.zeros((6, 6))
  for i in range(6):
    for j in range(6):
      ret[i, j] = C[i, j].subs([(q[i], qVal[i]) for i in range(6)] + [(dq[i], dqVal[i]) for i in range(6)]).evalf()
  return ret

def gnum(qVal):
  ret = np.zeros(6)
  for i in range(6):
    ret[i] = g[i].subs([(q[i], qVal[i]) for i in range(6)]).evalf()
  return ret

In [ ]:
sp.Array(Mnum([i + 1 for i in range(6)]))

[[10.9651232516462, 1.71588168464135, 0.701053802072838, 5.44776419721231, 2.70496619099055, 0.403610191040904], [1.71588168464135, 10.22187886919, 11.2686545434888, 1.22436982533715, 4.41543950944622, 0.912155126166288], [0.701053802072838, 11.2686545434888, 17.5654302177876, -0.0166743438293366, 4.56984845967482, 0.912155126166288], [5.44776419721231, 1.22436982533715, -0.0166743438293366, 7.01061934633536, 2.24488289648223, 1.47233108508257], [2.70496619099055, 4.41543950944622, 4.56984845967482, 2.24488289648223, 3.96342708199957, 0.68075478845144], [0.403610191040904, 0.912155126166288, 0.912155126166288, 1.47233108508257, 0.68075478845144, 1.0]]

In [ ]:
sp.Array(Cnum([i + 1 for i in range(6)], [-i - 1 for i in range(6)]))

In [ ]:
sp.Array(gnum([i + 1 for i in range(6)]))

[0.0, -25.6771341979879, -65.8180691052077, 11.7925752320015, -9.37578629853534, 0.0]